In [17]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import cv2
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tqdm import tqdm

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [18]:
#warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
input_video_name='test_video'
PATH_TO_SAVED_MODEL="exported-models/my_model/saved_model"
PATH_TO_LABELS="annotations/label_map.pbtxt"
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,use_display_name=True)

In [19]:
cap = cv2.VideoCapture(input_video_name+".mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

In [20]:
size

(1920, 1080)

In [21]:
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter(input_video_name+"_out"+'.mp4', fourcc, fps, size)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out=cv2.VideoWriter('Video.mp4',fourcc, fps, size)

In [22]:
progress = tqdm(total=length)
frame_number=0 
while cap.isOpened():
    progress.update(1)
    frame_number+=1
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    frame[:,:, [2, 0]] = frame[:,:, [0, 2]] 
    input_tensor = tf.convert_to_tensor(frame)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections = frame.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    image_np_with_detections[:,:, [2, 0]] = image_np_with_detections[:,:, [0, 2]] 
    gray_show = cv2.resize(image_np_with_detections, size, interpolation=cv2.INTER_CUBIC)
    #cv2.imshow('frame', gray_show)
    
    #print("Writing frame {} / {}".format(frame_number, length))
    out.write(gray_show)
    

    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

100%|██████████| 1824/1824 [02:10<00:00, 14.80it/s]

Can't receive frame (stream end?). Exiting ...


In [24]:
import os
os.system("ffmpeg -i test_video.mp4 -vcodec libx264 Video2.mp4")


0